# Import packages

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm
from preprocessing_functions import *

#optional
import datetime

# Read data path

In [2]:
!pwd

/ocean/projects/atm200007p/jlin96


In [21]:
def checktime():
    print(datetime.datetime.now()) 
data_path = "/ocean/projects/atm200007p/jlin96/longSPrun_o3/"
save_path = "/ocean/projects/atm200007p/jlin96/new_final_families/"

# Load data

In [4]:
sp_data = load_data(month = 9, year = 1, data_path = data_path)

In [5]:
sp_data

<xarray.Dataset>
Dimensions:       (time: 1440, lat: 64, lon: 128, crm_x: 32, crm_y: 1,
                   crm_z: 28, lev: 30, ilev: 31, isccp_prs: 7, isccp_tau: 7,
                   isccp_prstau: 49, tbnd: 2)
Coordinates:
  * lat           (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon           (lon) float64 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * crm_x         (crm_x) float64 0.0 4.0 8.0 12.0 ... 112.0 116.0 120.0 124.0
  * crm_y         (crm_y) float64 0.0
  * crm_z         (crm_z) float64 992.6 976.3 957.5 936.2 ... 38.27 24.61 14.36
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * isccp_prs     (isccp_prs) float64 90.0 245.0 375.0 500.0 620.0 740.0 900.0
  * isccp_tau     (isccp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 219.5
  * isccp_prstau  (isccp_prstau) float64 90.0 90.0 90.0 ... 900.0 900.0 900.2
  * time          (time) object 0001-09-01 00:00:00 ... 0001-09-30 23:30:00
Dimensions without coordinates: tbnd
Data variables: (12/43)
    P0            (time) float64 1e+05 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    time_bnds     (time, tbnd) object dask.array<chunksize=(48, 2), meta=np.ndarray>
    date_written  (time) |S8 dask.array<chunksize=(48,), meta=np.ndarray>
    time_written  (time) |S8 dask.array<chunksize=(48,), meta=np.ndarray>
    ntrm          (time) int32 63 63 63 63 63 63 63 63 ... 63 63 63 63 63 63 63
    ntrn          (time) int32 63 63 63 63 63 63 63 63 ... 63 63 63 63 63 63 63
    ...            ...
    RELHUM        (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    SHFLX         (time, lat, lon) float32 dask.array<chunksize=(48, 64, 128), meta=np.ndarray>
    SOLIN         (time, lat, lon) float32 dask.array<chunksize=(48, 64, 128), meta=np.ndarray>
    SPDQ          (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    SPDT          (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    TPHYSTND      (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    source:       CAM
    case:         AndKua_aqua_Base_training
    title:        
    logname:      jlin96
    host:         
    Version:      $Name:  $
    revision_Id:  $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $

# Get error weights

In [6]:
# outdated function I am still using for backwards compatibility

subsampling = 3

def sample_indices(size, spacing, fixed = True):
    numIndices = np.round(size/spacing)
    if fixed:
        indices = np.array([int(x) for x in np.round(np.linspace(1,size,int(numIndices)))])-1
    else:
        indices = list(range(size))
        np.random.shuffle(indices)
        indices = indices[0:int(numIndices)]
    return indices

In [17]:
# AREA WEIGHTING

lats = np.array(sp_data["lat"])
assert(90+lats[0]==90-lats[63])
last_lat_mdiff = 90+lats[0]
lat_mdiff = np.diff(lats)/2
lat_buff = np.append(lat_mdiff, last_lat_mdiff)
lat_edges = lat_buff + lats
lat_edges = np.append(-90, lat_edges)
area_weights = np.diff(np.sin(lat_edges*np.pi/180))[np.newaxis, np.newaxis,:,np.newaxis]

# PRESSURE WEIGHTING

pressure_grid_p1 = (sp_data['P0'].values[:, np.newaxis] * sp_data['hyai'].values)[:,:,np.newaxis, np.newaxis]
pressure_grid_p2 = sp_data['hybi'].values[:,:,np.newaxis,np.newaxis]*sp_data['NNPS'].values[:,np.newaxis,:,:]
pressure_grid = pressure_grid_p1 + pressure_grid_p2
dp = pressure_grid[:,1:31,:,:] - pressure_grid[:,0:30,:,:]
dp = dp[1:-1,:,:,:] # remove first and last timesteps in accordance with other preprocessing
dp = dp[0:336,:,:,:] # save only the first week
dp = dp[sample_indices(336, subsampling), :,:,:] # subsample every 3rd timestep (yes I know this was a bad way to do it)

# ERROR WEIGHTS

offline_error_weights = dp*area_weights

offline_error_weights_strato = offline_error_weights
offline_error_weights = offline_error_weights_strato[:,12:30,:,:]

offline_error_weights_strato = offline_error_weights_strato/np.sum(offline_error_weights_strato)
offline_error_weights_strato = np.transpose(offline_error_weights_strato, (1,0,2,3))

offline_error_weights = offline_error_weights/np.sum(offline_error_weights)
offline_error_weights = np.transpose(offline_error_weights, (1,0,2,3))

In [18]:
offline_error_weights_strato.shape

(30, 112, 64, 128)

In [19]:
offline_error_weights.shape

(18, 112, 64, 128)

In [28]:
np.sum(offline_error_weights_strato)

1.0000000000000038

In [29]:
np.sum(offline_error_weights)

1.0000000000000029

# Save error weights

In [22]:
with open(save_path + "offline_error_weights_strato.npy", 'wb') as f:
    np.save(f, np.float32(offline_error_weights_strato))

In [23]:
with open(save_path + "offline_error_weights.npy", 'wb') as f:
    np.save(f, np.float32(offline_error_weights))

In [24]:
print("finished")

finished
